# Querying Knowledge Graphs with Cypher

### Import packages and set up Neo4

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd

from langchain_community.graphs import Neo4jGraph

_ = load_dotenv()

### Setting Environment Variables

In [ ]:
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")
os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")

In [ ]:
kg = Neo4jGraph()

In [ ]:
kg.refresh_schema()
print(kg.schema)

### Examples to show why Knowledge Graphs

**Example of Schema Flexibility**

Before

In [ ]:
cypher = """
MATCH (game:Game {name:"Hades"})
RETURN game
"""

kg.query(cypher)

In [ ]:
cypher = """
CREATE (game:Game {name:"Hades"})
RETURN game
"""

kg.query(cypher)

After

In [ ]:
cypher = """
MATCH (game:Game {name:"Hades"})
RETURN game
"""

kg.query(cypher)

Before giving it new properties, we will formulate relationship.

In [ ]:
cypher = """
MATCH (hades:Game {name:"Hades"}), (rolePlaying:Genre {name:"Role-Playing"})
MERGE (hades)-[hasRelationship:BELONGS_TO_GENRE]->(rolePlaying)
RETURN hades, hasRelationship, rolePlaying
"""
kg.query(cypher)

In [ ]:
cypher = """
MATCH (n:Game {name:"Hades"})
SET n.release_year = 2020,
    n.genre = "Role-Playing",
    n.developer = "Supergiant Games"
"""
kg.query(cypher)

**Example of Query Simplicity and Readability**

Graph Example

In [ ]:
cypher = """
MATCH (g:Game)-[:BELONGS_TO_GENRE]->(genre:Genre), 
      (g)-[:PUBLISHED_BY]->(publisher:Publisher)
WHERE publisher.name = 'Nintendo'
RETURN genre.name, SUM(g.global_sales) AS TotalSales
ORDER BY TotalSales DESC
"""
kg.query(cypher)

SQL Example

In [ ]:
"""SELECT genre.name, SUM(g.global_sales) AS TotalSales
FROM games g
JOIN genres genre ON g.genre_id = genre.id
JOIN publishers publisher ON g.publisher_id = publisher.id
WHERE publisher.name = 'Nintendo'
GROUP BY genre.name
ORDER BY TotalSales DESC;"""

Display relationship between all nodes

In [ ]:
# MATCH p=()-[]->() RETURN p; Run in cypher shell

Delete all the nodes

In [ ]:
# MATCH (n)
# DETACH DELETE n